# Setup Database e Tabelas - MKL Bank

Este notebook cria o database, schema e todas as tabelas necessárias para o sistema bancário MKL baseado nos schemas dos arquivos CSV da pasta `sample_sintetic_data`.

## Objetivo
- Criar o database FSI (Financial Services Industry)
- Criar o schema de destino
- Analisar os arquivos CSV para definir estruturas de tabelas
- Criar todas as tabelas com os tipos de dados apropriados
- Verificar a criação das tabelas

In [ ]:
# Import Required Libraries
import os
import pandas as pd
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import logging
from datetime import datetime
import numpy as np

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# Database Configuration Setup
DB_CONFIG = {
    'host': os.getenv("PGHOST", "localhost"),
    'port': os.getenv("PGPORT", "5432"),
    'user': os.getenv("PGUSER", "postgres"),
    'password': os.getenv("PGPASSWORD", "postgres"),
    'database': os.getenv("PGDATABASE", "postgres")  # Default database to connect initially
}

TARGETSCHEMA = os.getenv("TARGETSCHEMA", "public")
TARGET_DB = os.getenv("PGDATABASE", "fsi_db")

# Caminho para os arquivos CSV
DATA_PATH = "../assets/sample_sintetic_data/"

print(f"🔧 Configuração do Database:")
print(f"   Host: {DB_CONFIG['host']}")
print(f"   Port: {DB_CONFIG['port']}")
print(f"   User: {DB_CONFIG['user']}")
print(f"   Target Database: {TARGET_DB}")
print(f"   Target Schema: {TARGETSCHEMA}")
print(f"   Data Path: {DATA_PATH}")

In [ ]:
# Database Connection Functions
def get_default_connection():
    """Conecta ao database padrão para operações administrativas"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except Exception as e:
        logger.error(f"Erro ao conectar ao database padrão: {e}")
        raise

def get_target_connection():
    """Conecta ao database de destino"""
    try:
        target_config = DB_CONFIG.copy()
        target_config['database'] = TARGET_DB
        conn = psycopg2.connect(**target_config)
        return conn
    except Exception as e:
        logger.error(f"Erro ao conectar ao database de destino: {e}")
        raise

def execute_sql(connection, sql, fetch=False):
    """Executa SQL e retorna resultado se necessário"""
    try:
        cursor = connection.cursor()
        cursor.execute(sql)
        
        if fetch:
            result = cursor.fetchall()
            cursor.close()
            return result
        else:
            connection.commit()
            cursor.close()
            return True
            
    except Exception as e:
        logger.error(f"Erro ao executar SQL: {e}")
        raise

print("✅ Funções de conexão definidas!")

In [ ]:
# Create Target Database
def create_database():
    """Cria o database de destino se não existir"""
    try:
        logger.info(f"Verificando se database '{TARGET_DB}' existe...")
        
        # Conecta ao database padrão
        conn = get_default_connection()
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        
        # Verifica se o database já existe
        check_db_sql = "SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s"
        cursor = conn.cursor()
        cursor.execute(check_db_sql, (TARGET_DB,))
        exists = cursor.fetchone()
        
        if not exists:
            # Cria o database
            create_db_sql = f'CREATE DATABASE "{TARGET_DB}"'
            cursor.execute(create_db_sql)
            logger.info(f"✅ Database '{TARGET_DB}' criado com sucesso!")
        else:
            logger.info(f"ℹ️ Database '{TARGET_DB}' já existe.")
            
        cursor.close()
        conn.close()
        
        return True
        
    except Exception as e:
        logger.error(f"❌ Erro ao criar database: {e}")
        raise

# Executar criação do database
create_database()

In [ ]:
# Create Schema
def create_schema():
    """Cria o schema de destino se não existir"""
    try:
        logger.info(f"Criando schema '{TARGETSCHEMA}' no database '{TARGET_DB}'...")
        
        conn = get_target_connection()
        
        # Cria o schema
        create_schema_sql = f'CREATE SCHEMA IF NOT EXISTS "{TARGETSCHEMA}"'
        execute_sql(conn, create_schema_sql)
        
        logger.info(f"✅ Schema '{TARGETSCHEMA}' criado/verificado com sucesso!")
        
        conn.close()
        return True
        
    except Exception as e:
        logger.error(f"❌ Erro ao criar schema: {e}")
        raise

# Executar criação do schema
create_schema()

In [ ]:
# Analyze CSV File Schemas
def analyze_csv_files():
    """Analisa os arquivos CSV para determinar os schemas das tabelas"""
    
    csv_files = [
        'agencias.csv',
        'clientes.csv', 
        'contas.csv',
        'cartoes.csv',
        'chaves_pix.csv',
        'transacoes.csv'
    ]
    
    schemas = {}
    
    for csv_file in csv_files:
        try:
            file_path = os.path.join(DATA_PATH, csv_file)
            
            if not os.path.exists(file_path):
                logger.warning(f"⚠️ Arquivo não encontrado: {file_path}")
                continue
                
            # Lê uma amostra do arquivo para análise
            df = pd.read_csv(file_path, nrows=100)
            table_name = csv_file.replace('.csv', '')
            
            logger.info(f"📊 Analisando {csv_file} ({len(df)} amostras)")
            print(f"\nTabela: {table_name}")
            print(f"Colunas: {list(df.columns)}")
            
            # Analisa tipos de dados
            column_types = {}
            for col in df.columns:
                col_data = df[col].dropna()
                
                if col_data.empty:
                    column_types[col] = "TEXT"
                    continue
                
                # Determina tipo baseado no conteúdo e nome da coluna
                if col in ['id_cliente', 'codigo_agencia', 'id_conta', 'cartao_id', 'agencia_id']:
                    column_types[col] = "BIGINT"
                elif col in ['chave_pix_id']:
                    column_types[col] = "SERIAL"
                elif 'id' in col.lower() and 'transacao' in col.lower():
                    column_types[col] = "VARCHAR(30)"
                elif 'data_' in col or 'created_at' in col:
                    column_types[col] = "TIMESTAMP"
                elif col in ['saldo', 'valor', 'limite_credito', 'limite']:
                    column_types[col] = "DECIMAL(15,2)"
                elif col in ['cpf']:
                    column_types[col] = "VARCHAR(14)"
                elif col in ['numero_cartao']:
                    column_types[col] = "VARCHAR(16)"
                elif col in ['codigo_barras', 'codigo_boleto']:
                    column_types[col] = "VARCHAR(100)"
                elif col in ['telefone']:
                    column_types[col] = "VARCHAR(20)"
                elif col in ['email']:
                    column_types[col] = "VARCHAR(200)"
                elif col in ['nome', 'nome_agencia', 'estabelecimento', 'beneficiario', 'depositante']:
                    column_types[col] = "VARCHAR(200)"
                elif col in ['endereco', 'descricao']:
                    column_types[col] = "TEXT"
                elif col in ['cidade']:
                    column_types[col] = "VARCHAR(100)"
                elif col in ['estado']:
                    column_types[col] = "CHAR(2)"
                elif col in ['cep']:
                    column_types[col] = "VARCHAR(10)"
                elif col in ['status', 'tipo', 'genero']:
                    column_types[col] = "CHAR(1)"
                elif col in ['tipo_cartao', 'tipo_chave', 'tipo_transferencia', 'tipo_deposito', 'categoria', 'operacao']:
                    column_types[col] = "VARCHAR(50)"
                elif col in ['chave_pix', 'chave_pix_destino', 'chave_pix_origem', 'valor_chave']:
                    column_types[col] = "VARCHAR(200)"
                else:
                    # Análise automática para outros campos
                    sample_val = str(col_data.iloc[0]) if len(col_data) > 0 else ""
                    if len(sample_val) > 100:
                        column_types[col] = "TEXT"
                    elif len(sample_val) > 50:
                        column_types[col] = "VARCHAR(200)"
                    else:
                        column_types[col] = "VARCHAR(100)"
            
            schemas[table_name] = {
                'columns': column_types,
                'sample_data': df.head(3)
            }
            
            print(f"Schema definido: {column_types}")
            
        except Exception as e:
            logger.error(f"❌ Erro ao analisar {csv_file}: {e}")
    
    return schemas

# Executar análise
table_schemas = analyze_csv_files()
print(f"\n✅ Análise concluída! {len(table_schemas)} tabelas analisadas.")

In [ ]:
# Generate Table Creation Scripts
def generate_create_table_scripts(schemas):
    """Gera scripts SQL para criação das tabelas"""
    
    create_scripts = {}
    
    # Ordem de criação (tabelas com FK devem vir depois)
    table_order = ['agencias', 'clientes', 'contas', 'cartoes', 'chaves_pix', 'transacoes']
    
    for table_name in table_order:
        if table_name not in schemas:
            continue
            
        columns = schemas[table_name]['columns']
        
        # Monta o script CREATE TABLE
        script_lines = [f'CREATE TABLE IF NOT EXISTS "{TARGETSCHEMA}".{table_name} (']
        
        column_definitions = []
        
        for col_name, col_type in columns.items():
            definition = f'    "{col_name}" {col_type}'
            
            # Adiciona constraints
            if col_name in ['id_cliente', 'codigo_agencia', 'id_conta', 'numero_cartao', 'id_transacao', 'chave_pix_id']:
                definition += ' PRIMARY KEY'
            elif col_name == 'cpf':
                definition += ' UNIQUE NOT NULL'
            elif 'id' in col_name.lower() and col_name != 'chave_pix_id':
                definition += ' NOT NULL'
            elif col_name in ['nome', 'chave_pix']:
                definition += ' NOT NULL'
                
            column_definitions.append(definition)
        
        # Adiciona foreign keys
        if table_name == 'contas':
            column_definitions.append(f'    FOREIGN KEY ("codigo_agencia") REFERENCES "{TARGETSCHEMA}".agencias("codigo_agencia")')
            column_definitions.append(f'    FOREIGN KEY ("id_cliente") REFERENCES "{TARGETSCHEMA}".clientes("id_cliente")')
        elif table_name == 'cartoes':
            column_definitions.append(f'    FOREIGN KEY ("id_cliente") REFERENCES "{TARGETSCHEMA}".clientes("id_cliente")')
        elif table_name == 'chaves_pix':
            column_definitions.append(f'    FOREIGN KEY ("id_cliente") REFERENCES "{TARGETSCHEMA}".clientes("id_cliente")')
        elif table_name == 'transacoes':
            column_definitions.append(f'    FOREIGN KEY ("id_conta") REFERENCES "{TARGETSCHEMA}".contas("id_conta")')
        
        script_lines.append(',\n'.join(column_definitions))
        script_lines.append(');')
        
        create_scripts[table_name] = '\n'.join(script_lines)
        
        logger.info(f"📝 Script gerado para tabela: {table_name}")
    
    return create_scripts

# Gerar scripts
table_scripts = generate_create_table_scripts(table_schemas)

# Exibir scripts gerados
print("📋 Scripts de criação de tabelas:")
print("=" * 60)
for table_name, script in table_scripts.items():
    print(f"\n-- Tabela: {table_name}")
    print(script)

In [ ]:
# Create Database Schema and Tables
def create_tables(scripts):
    """Executa os scripts de criação das tabelas"""
    
    try:
        conn = get_target_connection()
        
        logger.info("🚀 Iniciando criação das tabelas...")
        
        # Ordem de criação
        table_order = ['agencias', 'clientes', 'contas', 'cartoes', 'chaves_pix', 'transacoes']
        
        for table_name in table_order:
            if table_name in scripts:
                logger.info(f"📝 Criando tabela: {table_name}")
                
                try:
                    execute_sql(conn, scripts[table_name])
                    logger.info(f"✅ Tabela '{table_name}' criada com sucesso!")
                    
                except Exception as e:
                    logger.error(f"❌ Erro ao criar tabela '{table_name}': {e}")
                    raise
        
        conn.close()
        logger.info("🎉 Todas as tabelas foram criadas com sucesso!")
        
    except Exception as e:
        logger.error(f"❌ Erro na criação das tabelas: {e}")
        raise

# Executar criação das tabelas
create_tables(table_scripts)

In [ ]:
# Verify Table Creation
def verify_tables():
    """Verifica se todas as tabelas foram criadas corretamente"""
    
    try:
        conn = get_target_connection()
        
        # Lista todas as tabelas no schema
        list_tables_sql = """
            SELECT table_name, table_type
            FROM information_schema.tables 
            WHERE table_schema = %s
            ORDER BY table_name
        """
        
        cursor = conn.cursor()
        cursor.execute(list_tables_sql, (TARGETSCHEMA,))
        tables = cursor.fetchall()
        
        print(f"\n🔍 VERIFICAÇÃO DE TABELAS NO SCHEMA '{TARGETSCHEMA}'")
        print("=" * 60)
        
        if tables:
            print(f"📊 Total de tabelas encontradas: {len(tables)}")
            print("\nTabelas criadas:")
            
            for table_name, table_type in tables:
                print(f"✅ {table_name} ({table_type})")
                
                # Verifica colunas de cada tabela
                columns_sql = """
                    SELECT column_name, data_type, is_nullable, column_default
                    FROM information_schema.columns 
                    WHERE table_schema = %s AND table_name = %s
                    ORDER BY ordinal_position
                """
                
                cursor.execute(columns_sql, (TARGETSCHEMA, table_name))
                columns = cursor.fetchall()
                
                print(f"   📋 Colunas ({len(columns)}):")
                for col_name, data_type, nullable, default in columns:
                    null_info = "NULL" if nullable == "YES" else "NOT NULL"
                    default_info = f", DEFAULT: {default}" if default else ""
                    print(f"      • {col_name}: {data_type} ({null_info}{default_info})")
                print()
        else:
            print("⚠️ Nenhuma tabela encontrada!")
        
        cursor.close()
        conn.close()
        
        return len(tables)
        
    except Exception as e:
        logger.error(f"❌ Erro na verificação das tabelas: {e}")
        raise

# Executar verificação
table_count = verify_tables()

## ✅ Setup Concluído com Sucesso!

### 📊 Resumo da Execução:
- **Database:** `{TARGET_DB}` criado/verificado
- **Schema:** `{TARGETSCHEMA}` criado/verificado  
- **Tabelas:** {table_count} tabelas criadas com sucesso

### 🗂️ Estrutura Criada:
1. **agencias** - Dados das agências bancárias
2. **clientes** - Informações dos clientes
3. **contas** - Contas bancárias dos clientes
4. **cartoes** - Cartões de débito/crédito
5. **chaves_pix** - Chaves PIX cadastradas
6. **transacoes** - Transações financeiras

### 🚀 Próximos Passos:
1. **Execute o notebook:** `02_insert_main_data.ipynb` para inserir os dados principais
2. **Execute o notebook:** `03_insert_movement_data.ipynb` para inserir os dados de movimentações

### 📋 Observações:
- Todas as tabelas foram criadas com relacionamentos (Foreign Keys)
- Os tipos de dados foram definidos com base na análise dos CSVs
- Primary Keys e constraints foram aplicadas adequadamente
- O sistema está pronto para receber os dados!